Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Agapitov Denis"
COLLABORATORS = "Potapov Daniel"

---

# Домашнее задание №4. Матричная факторизация

## Задачи - 25 баллов (+5 доп баллов)
1. Ноутбук `mf.ipynb` - 20 баллов
- ImplicitALS - 4 балла
- SVD - 4 балла
- Dataset with features - 2 балла
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.

Используемые библиотеки в рамках ДЗ
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```

## Импорты и данные

In [ ]:
import os
import os.path
import threadpoolctl
import requests

import numpy as np
import pandas as pd
import zipfile as zf

from tqdm.auto import tqdm
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

In [3]:
data_path = os.environ.get("DATA_PATH")

Если вдруг у вас нет данных, то используйте закомментированный код

In [ ]:
# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [ ]:
if data_path is None:
    data_path = "data_original"  # ваш путь к данным до папки data_original включительно


interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

print(interactions.shape)
interactions.head(5)

In [13]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()

# dataset = Dataset.construct(train)

<ipython-input-13-1c199764f35c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)


In [ ]:
K_RECOS = 10
RANDOM_STATE = 49
NUM_THREADS = 0
map10 = MAP(k=K_RECOS)

## ImplicitALS

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях.

Правила пари:
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [16]:
# train preprocessing...
# YOUR CODE HERE
train2 = train.copy()
train2['watched_pct'] = train2['watched_pct'].fillna(0)
train2['weight'] = train2['watched_pct']
dataset = Dataset.construct(train2)

config = {
    'cls': 'ImplicitALSWrapperModel',
    'model': {
        'factors': 16,
        'regularization': 0.01,
        'num_threads': NUM_THREADS,
        'random_state': RANDOM_STATE
    }
}

Про конфиг, отличия ImplicitALSWrapperModel и PureSVDModel

https://chatgpt.com/share/67ed5427-4aa8-8004-b475-646d2f2a5c90

In [17]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'

model = model_from_config(config)
model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.052

0.05425174096711525
CPU times: user 2min 30s, sys: 3.06 s, total: 2min 33s
Wall time: 1min 37s


## SVD

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид.

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [18]:
import itertools

def svd_params_tuning():
    pct_limits = [0, 10, 15, 30]
    n_bins_list = [100, 50, 20, 10, 5]
    factors_list = [2, 4, 6, 8]
    variants = [1, 2]

    best_map = -np.inf
    best_params = None

    for pct_limit, factors in itertools.product(pct_limits, factors_list):
        variant = 1
        train2 = train.loc[
            ((train['watched_pct'] > pct_limit) & (train.groupby('user_id').transform('count')['item_id'] > 1))
            | (train['user_id'].isin(hot_users.tolist()))
        ].copy()

        dataset2 = Dataset.construct(train2)
        config = {
            'cls': 'PureSVDModel',
            'factors': factors,
            'recommend_n_threads': NUM_THREADS,
            'random_state': RANDOM_STATE
        }
        model = model_from_config(config)
        model.fit(dataset2)

        recos = model.recommend(
            users=hot_users,
            dataset=dataset2,
            k=K_RECOS,
            filter_viewed=True,
        )
        score = map10.calc(recos, test)
        print(f"Variant: {variant}, pct_limit: {pct_limit}, factors: {factors} -> map10: {score}")

        if score > best_map:
            best_map = score
            best_params = (variant, pct_limit, None, factors)

    for pct_limit, n_bins, factors in itertools.product(pct_limits, n_bins_list, factors_list):
        variant = 2
        train2 = train.loc[
            ((train['watched_pct'] > pct_limit) & (train.groupby('user_id').transform('count')['item_id'] > 1))
            | (train['user_id'].isin(hot_users.tolist()))
        ].dropna(subset='watched_pct').copy()

        train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

        dataset2 = Dataset.construct(train2)
        config = {
            'cls': 'PureSVDModel',
            'factors': factors,
            'recommend_n_threads': NUM_THREADS,
            'random_state': RANDOM_STATE
        }
        model = model_from_config(config)
        model.fit(dataset2)

        recos = model.recommend(
            users=hot_users,
            dataset=dataset2,
            k=K_RECOS,
            filter_viewed=True,
        )
        score = map10.calc(recos, test)
        print(f"Variant: {variant}, pct_limit: {pct_limit}, n_bins: {n_bins}, factors: {factors} -> map10: {score}")

        if score > best_map:
            best_map = score
            best_params = (variant, pct_limit, n_bins, factors)

    print(f"Best variant: {best_params[0]}, pct_limit: {best_params[1]}, n_bins: {best_params[2]}, factors: {best_params[3]} with map10: {best_map}")

In [19]:
# svd_params_tuning()

In [20]:
# train preprocessing...
# YOUR CODE HERE
pct_limit = 0
n_bins = 100

train2 = train.loc[
    ((train['watched_pct'] > pct_limit))
    | (train['user_id'].isin(hot_users.tolist()))
].dropna(subset='watched_pct').copy()

train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

dataset2 = Dataset.construct(train2)

config = {
    'cls': 'PureSVDModel',
    'factors': 2,
    'recommend_n_threads': NUM_THREADS,
    'random_state': RANDOM_STATE
}

In [21]:
%%time
assert config['cls'] == 'PureSVDModel'

model = model_from_config(config)
model.fit(dataset2)

recos = model.recommend(
    users=hot_users,
    dataset=dataset2,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.066

0.06829062961590548
CPU times: user 23.8 s, sys: 974 ms, total: 24.8 s
Wall time: 20.4 s


## Dataset with features

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [22]:
display(users.head())

user_features_frames = []
for feature in ["sex", "age", "income", "kids_flg"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
display(user_features.head())

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


,id,value,feature
0,973171,М,sex
1,962099,М,sex
2,1047345,Ж,sex
3,721985,Ж,sex
4,704055,Ж,sex


In [23]:
display(items.head())

items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"


content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))
display(item_features.head())

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [24]:
# YOUR CODE HERE
n_bins = 100
pct_limit = 0

train2 = train.loc[
    ((train['watched_pct'] > pct_limit))
    | (train['user_id'].isin(hot_users.tolist()))
].dropna(subset='watched_pct').copy()

train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)


dataset_with_features = Dataset.construct(
    interactions_df=train2,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income", "kids_flg"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [25]:
assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features

Собрав датасет с фичами вы готовы к третьему раунду пари.

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [26]:
import itertools

def als_params_tuning():
    pct_limits = [0, 10, 15]
    n_bins_list = [100, 50, 20, 10, 5]
    factors_list = [4, 6, 8]
    regularization_list = [0.01, 0.1]

    best_map = -np.inf
    best_params = None

    for pct_limit, n_bins, factors, reg in itertools.product(pct_limits, n_bins_list, factors_list, regularization_list):
        train2 = train.loc[
            (train['watched_pct'] > pct_limit)
            | (train['user_id'].isin(hot_users.tolist()))
        ].dropna(subset='watched_pct').copy()
        train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

        dataset_with_features = Dataset.construct(
            interactions_df=train2,
            user_features_df=user_features,
            cat_user_features=["sex", "age", "income", "kids_flg"],
            item_features_df=item_features,
            cat_item_features=["genre", "content_type"],
        )

        config = {
            'cls': 'ImplicitALSWrapperModel',
            'model': {
                'factors': factors,
                'regularization': reg,
                'num_threads': NUM_THREADS,
                'random_state': RANDOM_STATE
            }
        }

        model = model_from_config(config)
        model.fit(dataset_with_features)

        recos = model.recommend(
            users=hot_users,
            dataset=dataset_with_features,
            k=K_RECOS,
            filter_viewed=True,
        )
        score = map10.calc(recos, test)

        if score > best_map:
            best_map = score
            best_params = (pct_limit, n_bins, factors, reg)

        print(f"pct_limit: {pct_limit}, n_bins: {n_bins}, factors: {factors}, reg: {reg} -> map10: {score}")

    print(f"Best variant, pct_limit: {best_params[0]}, n_bins: {best_params[1]}, factors: {best_params[2]}, reg: {best_params[3]} with map10: {best_map}")

In [ ]:
# als_params_tuning()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

pct_limit: 0, n_bins: 100, factors: 4, reg: 0.01 -> map10: 0.07603246042619384


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

pct_limit: 15, n_bins: 5, factors: 8, reg: 0.01 -> map10: 0.07009616617487427


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

pct_limit: 15, n_bins: 5, factors: 8, reg: 0.1 -> map10: 0.07011096729940836
Best variant, pct_limit: 0, n_bins: 50, factors: 4, reg: 0.1 with map10: 0.07723829929236006


In [27]:
# YOUR CODE HERE
pct_limit = 0
n_bins = 50
factors = 4
reg = 0.1

train2 = train.loc[
    (train['watched_pct'] > pct_limit)
    | (train['user_id'].isin(hot_users.tolist()))
].dropna(subset='watched_pct').copy()
train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

dataset_with_features = Dataset.construct(
    interactions_df=train2,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income", "kids_flg"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

config = {
    'cls': 'ImplicitALSWrapperModel',
    'model': {
        'factors': factors,
        'regularization': reg,
        'num_threads': NUM_THREADS,
        'random_state': RANDOM_STATE
    }
}

model = model_from_config(config)
model.fit(dataset_with_features)

/usr/local/lib/python3.11/dist-packages/rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.073

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0.07723829929236006
CPU times: user 2min 2s, sys: 4.57 s, total: 2min 7s
Wall time: 1min 18s


## LightFM with features

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего старшего коллегу, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [29]:
import itertools

def lightfm_params_tuning():
    pct_limits = [0, 10, 15]
    n_bins_list = [100, 50, 20, 10, 5]
    factors_list = [4, 8, 16, 32]
    loss = 'warp'

    best_map = -np.inf
    best_params = None

    for pct_limit, n_bins, factors in itertools.product(pct_limits, n_bins_list, factors_list):
        train2 = train.loc[
            (train['watched_pct'] > pct_limit)
            | (train['user_id'].isin(hot_users.tolist()))
        ].dropna(subset='watched_pct').copy()
        train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

        dataset_with_features = Dataset.construct(
            interactions_df=train2,
            user_features_df=user_features,
            cat_user_features=["sex", "age", "income", "kids_flg"],
            item_features_df=item_features,
            cat_item_features=["genre", "content_type"],
        )

        config = {
            'cls': 'LightFMWrapperModel',
            'model': {
                'no_components': factors,
                'loss': loss,
                'random_state': RANDOM_STATE
            }
        }
        model = model_from_config(config)
        model.fit(dataset_with_features)

        recos = model.recommend(
            users=hot_users,
            dataset=dataset_with_features,
            k=K_RECOS,
            filter_viewed=True,
        )

        score = map10.calc(recos, test)

        if score > best_map:
            best_map = score
            best_params = (pct_limit, n_bins, factors)

        print(f"pct_limit: {pct_limit}, n_bins: {n_bins}, factors: {factors} -> map10: {score}")

    print(f"Best variant, pct_limit: {best_params[0]}, n_bins: {best_params[1]}, factors: {best_params[2]} with map10: {best_map}")

In [33]:
# lightfm_params_tuning()

pct_limit: 0, n_bins: 100, factors: 4 -> map10: 0.08055969293639234
pct_limit: 0, n_bins: 100, factors: 8 -> map10: 0.08042370041216466
pct_limit: 0, n_bins: 100, factors: 16 -> map10: 0.07967380748461277
pct_limit: 0, n_bins: 100, factors: 32 -> map10: 0.07998433233794912
pct_limit: 0, n_bins: 50, factors: 4 -> map10: 0.07825938841290668
pct_limit: 0, n_bins: 50, factors: 8 -> map10: 0.07825546939363832
pct_limit: 0, n_bins: 50, factors: 16 -> map10: 0.0805410253312979
pct_limit: 0, n_bins: 50, factors: 32 -> map10: 0.0789592481421267
pct_limit: 0, n_bins: 20, factors: 4 -> map10: 0.08161419207765262
pct_limit: 0, n_bins: 20, factors: 8 -> map10: 0.08052143307696084
pct_limit: 0, n_bins: 20, factors: 16 -> map10: 0.07947491036116981
pct_limit: 0, n_bins: 20, factors: 32 -> map10: 0.07976609904515941
pct_limit: 0, n_bins: 10, factors: 4 -> map10: 0.07939559308255094
pct_limit: 0, n_bins: 10, factors: 8 -> map10: 0.08022850140386864
pct_limit: 0, n_bins: 10, factors: 16 -> map10: 0.0779

In [30]:
# YOUR CODE HERE
pct_limit = 0
n_bins = 20
factors = 4
loss = 'warp'

train2 = train.loc[
    (train['watched_pct'] > pct_limit)
    | (train['user_id'].isin(hot_users.tolist()))
].dropna(subset='watched_pct').copy()
train2['weight'] = pd.cut(train2['watched_pct'], bins=n_bins+1, labels=False)

dataset_with_features = Dataset.construct(
    interactions_df=train2,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income", "kids_flg"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

config = {
    'cls': 'LightFMWrapperModel',
    'model': {
        'no_components': factors,
        'loss': loss,
        'random_state': RANDOM_STATE
    }
}

In [31]:
%%time
assert config['cls'] == 'LightFMWrapperModel'

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.08

0.08161419207765262
CPU times: user 26.5 s, sys: 235 ms, total: 26.7 s
Wall time: 26.8 s


## Сервис

Эта битва была легендарной, но вот за спиной опять появляется ваш продакт.

"Время катить АБ" - говорит он в пятницу вечером. Делать нечего, вы собираете ваши наработки и определяете совместный фронт работ.

В прод должна заехать лучшая модель, которая побьет текущей модели в проде в `MAP@10 = 0.075`.

Также есть бонус от вашего старшего коллеги, который вам советует присмотреться к `Approximate nearest neighbours`, например к `nmslib`.

"Если сможешь обернуть в ANN, то на следующем годовом ревью получишь от меня оценку отлично" - сказал он. Изучить новые технологии и получить повышение - идеально, заключаете вы и бросаетесь в бой.

# Вместо заключения

## Задачи - 25 баллов (30 баллов с доп задачей по ANN)
1. Ноутбук `mf.ipynb` - 20 баллов
- SVD - 5 баллов
- ImplicitALS - 5 баллов
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.